In [3]:
%pip install httpx

Note: you may need to restart the kernel to use updated packages.


In [4]:
import httpx

# Comparing prod and dev

Ok, not much difference here.

In [5]:
prod = "https://wersdoerfer.de"
staging = "https://homepage.staging.django-cast.com"
dev = "http://localhost:8000"
blog_slug = "ephes_blog"
# blog_slug = "das_claas_und_nora_blog"

In [6]:
%%time
r = httpx.get(f"{prod}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

72089
CPU times: user 25.9 ms, sys: 7.8 ms, total: 33.7 ms
Wall time: 262 ms


In [7]:
%%time
r = httpx.get(f"{staging}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

60367
CPU times: user 28.2 ms, sys: 4.99 ms, total: 33.2 ms
Wall time: 363 ms


In [8]:
%%time
r = httpx.get(f"{dev}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

89084
CPU times: user 6.71 ms, sys: 2.11 ms, total: 8.81 ms
Wall time: 328 ms


In [9]:
%%time
r = httpx.head(f"{dev}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

0
CPU times: user 19.5 ms, sys: 3.45 ms, total: 23 ms
Wall time: 229 ms


In [1]:
print(f"{dev}/blogs/{blog_slug}/")

NameError: name 'dev' is not defined

# What About the Feed?

In [11]:
%%time
print("Production!")
r = httpx.get(f"{prod}/blogs/{blog_slug}/feed/rss.xml")
r.raise_for_status()
content = r.text
print(len(content))

Production!


ReadTimeout: The read operation timed out

In [12]:
%%time
print("Staging!")
r = httpx.get(f"{staging}/blogs/{blog_slug}/feed/rss.xml", timeout=600)
r.raise_for_status()
content = r.text
print(len(content))

Staging!
1937352
CPU times: user 33.7 ms, sys: 10.6 ms, total: 44.3 ms
Wall time: 48.4 s


In [10]:
%%time
print("Development!")
r = httpx.get(f"{dev}/blogs/{blog_slug}/feed/rss.xml")
r.raise_for_status()
content = r.text
print(len(content))

Development!
1976775
CPU times: user 24.1 ms, sys: 8.06 ms, total: 32.2 ms
Wall time: 1.42 s


In [11]:
print(blog_slug)

ephes_blog


## Claas Blog

In [13]:
blog_slug = "das_claas_und_nora_blog"

In [8]:
%%time
print("Development!")
r = httpx.get(f"{dev}/blogs/{blog_slug}/feed/rss.xml")
r.raise_for_status()
content = r.text
print(len(content))

Development!
1968423
CPU times: user 14.4 ms, sys: 5.94 ms, total: 20.3 ms
Wall time: 2.87 s


In [15]:
%%time
print("Staging!")
r = httpx.get(f"{staging}/blogs/{blog_slug}/feed/rss.xml", timeout=600)
r.raise_for_status()
content = r.text
print(len(content))

Staging!
4548301
CPU times: user 60.3 ms, sys: 55.7 ms, total: 116 ms
Wall time: 3min 51s


In [16]:
%%time
print("Production!")
r = httpx.get(f"{prod}/blogs/{blog_slug}/feed/rss.xml", timeout=600)
r.raise_for_status()
content = r.text
print(len(content))

Production!
4859899
CPU times: user 46.1 ms, sys: 35.6 ms, total: 81.7 ms
Wall time: 5min 22s


## What about the feed call?

In [18]:
from django.test.client import RequestFactory
from cast.feeds import LatestEntriesFeed

In [23]:
%%time
feed_view = LatestEntriesFeed()
request_factory = RequestFactory()
request = request_factory.get(f"/blogs/{blog_slug}/feed/rss.xml")
r = feed_view(request, slug=blog_slug)

CPU times: user 4.12 s, sys: 164 ms, total: 4.28 s
Wall time: 4.76 s


In [25]:
print(len(r.content))

4860770


# How long does the serve call take?

In [9]:
from django.test.client import RequestFactory
from django_htmx.middleware import HtmxDetails

In [10]:
request_factory = RequestFactory()
request = request_factory.get("/")
request.htmx = HtmxDetails(request)

In [11]:
blog = Blog.objects.get(slug=blog_slug)

In [12]:
%%time
response = blog.serve(request).render()
print(len(response.content))

59238
CPU times: user 265 ms, sys: 58.4 ms, total: 324 ms
Wall time: 389 ms


/Users/jochen/.virtualenvs/homepage/lib/python3.12/site-packages/wagtail/blocks/base.py:233: RemovedInWagtail60Warning: GalleryBlock.get_template should accept a 'value' argument as first argument
  warnings.warn(


In [10]:
[b.slug for b in Blog.objects.all()]

['das_claas_und_nora_blog', 'ephes_blog']

In [11]:
#Rendition.objects.all().delete()

In [45]:
slug = "weeknotes-2023-07-24"
post = Post.objects.get(slug=slug)

In [46]:
post.sync_media_ids()

In [47]:
post.media_ids_from_db

{'image': set(), 'video': set(), 'gallery': {228}, 'audio': set()}

In [48]:
post.media_ids_from_body

{'gallery': {228}}

In [50]:
[gallery] = post.galleries.all()

In [51]:
gallery


<Gallery: Gallery object (228)>

In [52]:
gallery.images.all()

<ImageQuerySet [<Image: Marktplatz in Vannes>, <Image: Fachwerkhäuser in Vannes>, <Image: Fensterspiegelung>, <Image: Sonnenstern an der Saline>, <Image: Fernglas zum Beobachten der Vögel an der Saline>, <Image: Salz in der Saline>, <Image: Windrad neben der Saline>, <Image: Ein paar Blumen am Wegesrand>, <Image: Atlantikküste vor Quiberon>, <Image: Steilküste vor Quiberon>, <Image: Strand in Quiberon>, <Image: Am Strand von Mousterian>, <Image: Frau am Strand von Mousterian>]>

In [53]:
post.images.all()

<ImageQuerySet []>

In [55]:
Image.objects.count()

1734

## Count Queries

In [26]:
import sqlparse

from django.db import connection, reset_queries

In [27]:
def count_queries(func):
    reset_queries()
    response = func()
    print(len(response.content))
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)    

In [29]:
feed_view = LatestEntriesFeed()
request_factory = RequestFactory()
request = request_factory.get(f"/blogs/{blog_slug}/feed/rss.xml")
func = lambda: feed_view(request, slug=blog_slug)
count_queries(func)

4860770


3220

In [15]:
request_factory = RequestFactory()
request = request_factory.get("/")
htmx


In [16]:
blog = Blog.objects.get(slug=blog_slug)

In [17]:
%%time
num_queries = count_queries(blog.serve(request).render)
print(num_queries)

5
CPU times: user 174 ms, sys: 22.5 ms, total: 196 ms
Wall time: 209 ms


In [31]:
# show_queries(connection.queries)